In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

In [24]:
lst = pd.read_csv("listings.csv")

Need to write a function that will tell me basics about each file in terms of:
- display the head info for each dataframe
- number of rows and columns
- which features are categorical vs numeric
- basic statistics about the numeric features
- unique values of the categorical features
- info relating to the missing data including how many features have missing data, proportion of missing by feature

In [54]:
def describe_data(df, csv_name):
    dash = '-' * 70
    newline = '\n' * 3
    
    #head info
    print(f'{dash}Head Information for {csv_name}{dash}')
    display(df.head())
    
    #basic shape and stats
    print(newline)
    print(f'{dash}Basic Shape Information for {csv_name}{dash}')
    print(f'{csv_name} contains {df.shape[0]} rows and {df.shape[1]} columns.')
    
    
    
    return

In [55]:
describe_data(lst,"listings.csv")

----------------------------------------------------------------------Head Information for listings.csv----------------------------------------------------------------------


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89






----------------------------------------------------------------------Basic Shape Information for listings.csv----------------------------------------------------------------------
listings.csv contains 3818 rows and 92 columns.
